In [ ]:
!pip install z3-solver

     |████████████████████████████████| 32.6MB 87kB/s 


In [ ]:
from z3 import *
import itertools
from google.colab import drive
import time

In [ ]:
drive.mount('/gdrive', force_remount=True)
%cd /gdrive/MyDrive/CDMO/instances/

Mounted at /gdrive
/gdrive/MyDrive/CDMO/instances


In [ ]:
filename = 'ins-32.txt'
with open(filename, "r") as file1:
    FileasList = file1.readlines()
W=int(FileasList[0])
N=int(FileasList[1])

In [ ]:
rects=[]
totarea=0
for n in range(2,2+N):
  values = [int(s) for s in FileasList[n].split() if s.isdigit()]
  w =values[0]
  h = values[1]
  totarea += h*w
  rects.append([w,h])

In [ ]:
m,coords = solve(rects,W, opt=True)

39
40
41


Exception ignored in: <function AstRef.__del__ at 0x7f74cdec24d0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/z3/z3.py", line 313, in __del__
    if self.ctx.ref() is not None and self.ast is not None:
  File "/usr/local/lib/python3.7/dist-packages/z3/z3.py", line 198, in ref
    return self.ctx
KeyboardInterrupt


KeyboardInterrupt: ignored

In [ ]:
def solve(rects,W,opt=False):
  start = time.time()
  solved = False
  H= int(totarea/W)
  while (solved != True):
    print(H)
    px = [[Bool(f"px{r+1}_{x}") for x in range(W)] for r in range(len(rects))]
    py = [[Bool(f"py{r+1}_{y}") for y in range(H)] for r in range(len(rects))]
    under = [[Bool(f"r{ri+1}_under_r{rj+1}") if ri != rj else 0 for rj in range(len(rects)) ] for ri in range(len(rects))]
    left = [[Bool(f"r{ri+1}_leftof_r{rj+1}")  if rj!=ri else 0 for rj in range(len(rects))] for ri in range(len(rects))]
  
    #add automatic truths
    s= Solver()
    for r in range(len(rects)):
      for xi in range(W-rects[r][0],W):
        s.add(px[r][xi])
      for yi in range(H-rects[r][1],H):
        s.add(py[r][yi])

    #add symm breaking and optimization
    if opt == True:

      #limit the dommain of the biggest rectangle, thus avoiding most symm options
      """bigger_r = max(rects, key=lambda p:p[0]*p[1])
      ri = rects.index(bigger_r)
      limit_x = int((W-bigger_r[0])/2)
      s.add(Or([px[ri][i] for i in range(limit_x+1)]))
      for r in range(len(rects)):
        if r!= ri and rects[r][0] > limit_x:
          s.add(Not(left[r][ri]))

      limit_y = int((H-bigger_r[1])/2)
      s.add(Or([py[ri][i] for i in range(limit_y+1)]))
      for r in range(len(rects)):
        if r!= ri and rects[r][1] > limit_y:
          s.add(Not(under[r][ri])) """

      
      #avoid combinations of rectangles that would go out of borders
      for (ri,rj) in itertools.combinations(range(len(rects)),2):
        if (rects[ri][0] + rects[rj][0] > W):
          s.add(Not(left[ri][rj]))
          s.add(Not(left[rj][ri]))
        if (rects[ri][1] + rects[rj][1] > H):
          s.add(Not(under[ri][rj]))
          s.add(Not(under[rj][ri]))

      """#fix positions for same size rectangles
      for (ri,rj) in itertools.combinations(range(len(rects)),2):
        if (rects[ri] == rects[rj]):
          s.add(Not(left[rj][ri]))
          s.add(Or(Not(under[rj][ri]),left[ri][rj]))"""

    #axioms
  
    for r in range(len(rects)):
      for e in range(W-1):
        s.add(Or(Not(px[r][e]),px[r][e+1]))
      for f in range(H-1):
        s.add(Or(Not(py[r][f]),py[r][f+1]))

    #non overlapping condition
    for (i,j) in itertools.combinations(range(len(rects)),2):
      s.add(Or(left[i][j],left[j][i],under[i][j],under[j][i]))
  
    #meaning for left and under
    for ri in range(len(rects)):
      for rj in range(len(rects)):
        if ri != rj :
          #print((ri,rj))
          s.add(Or(
              Not(left[ri][rj]),
              Not(px[rj][rects[ri][0]-1])
          )) # left(ri,rj) -> xj > wi, lower bound for xj
          for e in range(0,W-rects[ri][0]):
            s.add(Or(
                Not(left[ri][rj]),
                px[ri][e],
                Not(px[rj][e+rects[ri][0]])
            )) #for any e: left(ri,rj) -> (xi <e v ! xj<e+wi)
  
          s.add(Or(
              Not(under[ri][rj]),
              Not(py[rj][rects[ri][1]-1])
          )) #under(ri,rj)-> yj > hi, lower bound for yj
  
          for f in range(0,H-rects[ri][1]):
            s.add(Or(
                Not(under[ri][rj]),
                py[ri][f],
                Not(py[rj][f+rects[ri][1]])
          )) #for any f, under(ri,rj) -> (yj <= f+hi ->yi <= f)
    

    #solve
    if s.check() == z3.sat:
      solved = True
      m = s.model()
      print(f'H found: {H}')
      end=time.time()
      print(f"elapsed:{end-start}")
      return m,return_coods(m,len(rects),px,py)
    H +=1

In [ ]:
def return_coods(m,l,px,py):
  x_r =[]
  y_r=[]
  for r in range(l):
    x_r.append([m.eval(x) for x in px[r]].index(True))

  for r in range(len(rects)):
    y_r.append([m.eval(x) for x in py[r]].index(True))

  return list(zip(x_r,y_r))

In [ ]:
coords

[(0, 14),
 (7, 0),
 (22, 32),
 (25, 31),
 (26, 0),
 (22, 24),
 (0, 17),
 (0, 26),
 (7, 4),
 (34, 0),
 (0, 0),
 (22, 7),
 (30, 13),
 (7, 16),
 (22, 4),
 (22, 0),
 (3, 0),
 (25, 7),
 (3, 27),
 (3, 5),
 (33, 13),
 (29, 0),
 (25, 27),
 (28, 31),
 (29, 3),
 (25, 13),
 (10, 0)]

In [ ]:
m,coords = solve(rects,W, opt=False)

5
6
7
8
H found: 8
elapsed:0.20158839225769043


In [ ]:
coords

[(0, 5), (5, 0), (3, 5), (0, 0)]

In [ ]:
#adattata da quella di carlo
def plot_result(result):
    dims = result["dims_v"]
    w = result["w_v"]
    ys = np.array(result["sol"])[:,1]
    xs = np.array(result["sol"])[:,0]

    lims = (0, max(w, max([ys[i]+dims[i][1] for i in range(len(ys))])))

    fig1 = plt.figure(figsize=(10, 10))
    ax1 = fig1.add_subplot(111, aspect='equal')
    for i in range(len(xs)):
        ax1.add_patch(patches.Rectangle((xs[i], ys[i]), dims[i][0], dims[i][1], edgecolor='black', facecolor=random.choice(['r', 'g', 'y'])))

    plt.xticks(range(w+1))
    plt.yticks(range(max([ys[i]+dims[i][1] for i in range(len(ys))])+1))
    #plt.grid(b=True, which='major', color='#666666', linestyle='-')
    plt.show()

In [ ]:
coords

[(5, 0), (0, 3), (0, 0), (3, 3)]

In [ ]:
### DEBUG CODE FROM HERE
if True == False:
  print("how?")

In [ ]:
for t in m:
  if is_true(m[t]):
    print(t)

px1_7
py4_3
py2_3
px1_5
px3_5
py3_7
py4_7
px1_0
py4_6
r1_under_r4
r3_under_r2
px3_4
py2_6
px4_5
px4_6
py1_4
py3_6
py1_3
py1_6
r3_under_r4
px2_5
py2_5
px3_7
px4_3
px4_4
px3_1
py3_3
py3_2
py1_0
r1_under_r2
py3_5
px2_2
py2_4
px2_6
r3_leftof_r1
px3_2
py1_5
py2_7
px3_0
r2_leftof_r4
py3_1
px1_1
px3_3
px2_3
px3_6
py3_4
px4_7
py1_7
py4_5
py1_1
px2_1
py4_4
px2_4
py3_0
py1_2
px1_6
px2_7
px2_0


In [ ]:
W=8
H=10

In [ ]:
#TODO add the "certainly true px and py"
#px = [[Bool(f"x{r+1}<{x}") for x in range(W-rects[r][0]+1)] for r in range(len(rects))]
#py = [[Bool(f"y{r+1}<{y}") for y in range(H-rects[r][1]+1)] for r in range(len(rects))]
px = [[Bool(f"px{r+1}_{x}") for x in range(W)] for r in range(len(rects))]
py = [[Bool(f"py{r+1}_{y}") for y in range(H)] for r in range(len(rects))]

In [ ]:
under = [[Bool(f"r{ri+1}_under_r{rj+1}") if ri != rj else 0 for rj in range(len(rects)) ] for ri in range(len(rects))]
left = [[Bool(f"r{ri+1}_leftof_r{rj+1}")  if rj!=ri else 0 for rj in range(len(rects))] for ri in range(len(rects))]

In [ ]:
s= Solver()
#add automatic truths

for r in range(len(rects)):
  for xi in range(W-rects[r][0],W):
    s.add(px[r][xi])
  for yi in range(H-rects[r][1],H):
    s.add(py[r][yi])


In [ ]:
#axioms

for r in range(len(rects)):
  for e in range(W-1):
    s.add(Or(Not(px[r][e]),px[r][e+1]))
  for f in range(H-1):
    s.add(Or(Not(py[r][f]),py[r][f+1]))

In [ ]:

#non overlapping condition
for (i,j) in itertools.combinations(range(len(rects)),2):
  s.add(Or(left[i][j],left[j][i],under[i][j],under[j][i]))

In [ ]:
for ri in range(len(rects)):
  for rj in range(len(rects)):
    if ri != rj :
      #print((ri,rj))
      s.add(Or(
          Not(left[ri][rj]),
          Not(px[rj][rects[ri][0]-1])
      )) # left(ri,rj) -> xj > wi, lower bound for xj
      for e in range(0,W-rects[ri][0]):
        s.add(Or(
            Not(left[ri][rj]),
            px[ri][e],
            Not(px[rj][e+rects[ri][0]])
        )) #for any e: left(ri,rj) -> (xi <e v ! xj<e+wi)

      s.add(Or(
          Not(under[ri][rj]),
          Not(py[rj][rects[ri][1]-1])
      )) #under(ri,rj)-> yj > hi, lower bound for yj

      for f in range(0,H-rects[ri][1]):
        s.add(Or(
            Not(under[ri][rj]),
            py[ri][f],
            Not(py[rj][f+rects[ri][1]])
      )) #for any f, under(ri,rj) -> (yj <= f+hi ->yi <= f)

In [ ]:
s.check()

sat

In [ ]:
m = s.model()

In [ ]:
x_r =[]
y_r =[]
for r in range(len(rects)):
  x_r.append([m.eval(x) for x in px[r]].index(True))

y_r=[]
for r in range(len(rects)):
  y_r.append([m.eval(x) for x in py[r]].index(True))

In [ ]:
coords=list(zip(x_r,y_r))

In [ ]:
[m.eval(x) for x in px[0]]

[False, False, False, False, False, True, True, True]

In [ ]:
print(coords)

[(5, 0), (0, 3), (0, 0), (3, 3)]


In [ ]:
start = time.time()
solved = False
H=max(rects, key=lambda x:x[1])[1]
while (solved != True):
  print(H)
  px = [[Bool(f"px{r+1}_{x}") for x in range(W)] for r in range(len(rects))]
  py = [[Bool(f"py{r+1}_{y}") for y in range(H)] for r in range(len(rects))]
  under = [[Bool(f"r{ri+1}_under_r{rj+1}") if ri != rj else 0 for rj in range(len(rects)) ] for ri in range(len(rects))]
  left = [[Bool(f"r{ri+1}_leftof_r{rj+1}")  if rj!=ri else 0 for rj in range(len(rects))] for ri in range(len(rects))]
 
  #add automatic truths
  s= Solver()
  for r in range(len(rects)):
    for xi in range(W-rects[r][0],W):
      s.add(px[r][xi])
    for yi in range(H-rects[r][1],H):
      s.add(py[r][yi])
 
  #axioms
 
  for r in range(len(rects)):
    for e in range(W-1):
      s.add(Or(Not(px[r][e]),px[r][e+1]))
    for f in range(H-1):
      s.add(Or(Not(py[r][f]),py[r][f+1]))
 
  #non overlapping condition
  for (i,j) in itertools.combinations(range(len(rects)),2):
    s.add(Or(left[i][j],left[j][i],under[i][j],under[j][i]))
 
  #meaning for left and under
  for ri in range(len(rects)):
    for rj in range(len(rects)):
      if ri != rj :
        #print((ri,rj))
        s.add(Or(
            Not(left[ri][rj]),
            Not(px[rj][rects[ri][0]-1])
        )) # left(ri,rj) -> xj > wi, lower bound for xj
        for e in range(0,W-rects[ri][0]):
          s.add(Or(
              Not(left[ri][rj]),
              px[ri][e],
              Not(px[rj][e+rects[ri][0]])
          )) #for any e: left(ri,rj) -> (xi <e v ! xj<e+wi)
 
        s.add(Or(
            Not(under[ri][rj]),
            Not(py[rj][rects[ri][1]-1])
        )) #under(ri,rj)-> yj > hi, lower bound for yj
 
        for f in range(0,H-rects[ri][1]):
          s.add(Or(
              Not(under[ri][rj]),
              py[ri][f],
              Not(py[rj][f+rects[ri][1]])
        )) #for any f, under(ri,rj) -> (yj <= f+hi ->yi <= f)
  if s.check() == z3.sat:
    solved = True
    m = s.model()
    print(f'H found: {H}')
    end=time.time()
    print(f"elapsed:{end-start}")
  H +=1

5
6
7
8
H found: 8
elapsed:0.17603826522827148
